In [1]:
import io
import pandas as pd

In [2]:
def stream_filter(stream):
    xc3 = int.from_bytes(b'\xc3', byteorder='big')
    xc2 = int.from_bytes(b'\xc2', byteorder='big')
    m = int.from_bytes(b'\x40', byteorder="big")
    after_xc3 = False
    for s in stream:
        if s == xc3:
            after_xc3 = True
            continue            
        if after_xc3:
            after_xc3 = False
            yield s|m
            continue
        if s == xc2:
            continue
        yield s

In [3]:
def read_fix_emoji(path):
    with open(path, 'rb') as f:
        bytes_in = f.read()
    inmemory = io.BytesIO(bytes(stream_filter(bytes_in)))
    return pd.read_csv(inmemory, encoding_errors='ignore')

In [4]:
train_df = read_fix_emoji('train_tweets.csv')
train_df.to_csv('train_tweets_fix.csv', index=False)
test_df = read_fix_emoji('test_tweets.csv')
test_df.to_csv('test_tweets_fix.csv', index=False)